# Tutorial

## Motivation


Project efforts such as the [Coupled Model Intercomparison Project (CMIP)](https://www.wcrp-climate.org/wgcm-cmip) and the [Community Earth System Model (CESM) Large Ensemble Project](http://www.cesm.ucar.edu/projects/community-projects/LENS/) produce a huge of amount climate data persisted on tape, disk storage, object storage components across multiple (in the order of ~ 300,000) data assets. These data assets are stored in netCDF and more recently [Zarr](https://zarr.readthedocs.io/en/stable/) formats. Finding, investigating, loading these assets into data array containers such as xarray can be a daunting task due to the large number of files a user may be interested in. Intake-esm aims to address these issues by providing necessary functionality for searching, discovering, data access/loading. 

## Overview 

`intake-esm` is a data cataloging utility built on top of [intake](https://github.com/intake/intake), [pandas](https://pandas.pydata.org/), and [xarray](https://xarray.pydata.org/en/stable/), and it's pretty awesome! 


## Opening a collection

First step is to point ``intake-esm`` to an ESM (Earth System Model) collection definition file, which is a JSON file that conforms to the [ESM Collection Specification](https://github.com/NCAR/esm-collection-spec). The collection JSON file can be stored on a local filesystem or can be hosted on a remote server. When provided a link/path to an esm collection file, `intake-esm` establishes a link to a database (CSV file) that contains assets (e.g. file) locations and associated metadata (i.e., which experiement, model, the come from). 


For demonstration purposes, we will be using the CMIP6 data hosted in Pangeo's Google Storage. For this data collection, we will be using the Pangeo cloud collection file stored [here](https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json).


<div class="alert alert-info">

**Note:** 
    

More collection examples are available in [intake-esm-datastore](https://github.com/NCAR/intake-esm-datastore) GitHub repository. 
    
    
</div>


Because `intake-esm` is an `intake` plugin, the plugin automatically appears in the set of known
plugins in the intake registry, and an associated ``intake.open_esm_datastore`` function is created at import time.

In [ ]:
import intake

In [ ]:
intake.registry

Let's now open a collection catalog for CMIP6 data residing in Pangeo's Google storage. 

In [ ]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
col

Since `intake-esm` is build on top of [pandas](https://pandas.pydata.org/pandas-docs/stable), it is possible to view the `pandas.DataFrame` as follows.

In [ ]:
col.df.head()

It is possible to interact with the `DataFrame`; for instance, we can see what the "attributes" of the datasets are by printing the columns.

In [ ]:
col.df.columns

## Search and discovery

### Finding unique entries
Let's query the data to see what models ("source_id"), experiments ("experiment_id") and temporal frequencies ("table_id") are available.

In [ ]:
import pprint 
uni_dict = col.unique(['source_id', 'experiment_id', 'table_id'])
pprint.pprint(uni_dict, compact=True)

### Searching for specific datasets


One of the features supported in ``intake-esm`` is querying the collection catalog.
This feature is provided via the ``search()`` method. The ``search()`` method allows the user to
specify a query by using keyword arguments. This method returns a subset of the collection catalog
with all the entries that match the provided query.

Let's find all the dissolved oxygen data at annual frequency from the ocean for the `historical` and `ssp585` experiments.

In [ ]:
%%time
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Oyr', variable_id='o2', 
                 grid_label='gn')
cat.df

In [ ]:
len(cat.df), len(col.df)

You can get summary information by using `.nunique()` and `.unique()` methods:

In [ ]:
pprint.pprint(cat.unique(columns=["source_id", "activity_id"]), indent=4)

In [ ]:
cat.nunique()

## Loading data

The best part about `intake-esm` is that it enables loading data directly into an [xarray.Dataset](http://xarray.pydata.org/en/stable/api.html#dataset).

Note that data on the cloud are in 
[zarr](https://zarr.readthedocs.io/en/stable/) and data on 
[glade](https://www2.cisl.ucar.edu/resources/storage-and-file-systems/glade-file-spaces) are stored as 
[netCDF](https://www.unidata.ucar.edu/software/netcdf/) files. This is opaque to the user!

`intake-esm` has rules for aggegating datasets; these rules are defined in the collection-specification file.

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
                                cdf_kwargs={'chunks': {}, 'decode_times': False})

`dset_dict` is a dictionary of `xarray.Dataset`'s; its keys are constructed to refer to compatible groups.

In [ ]:
dset_dict.keys()

We can access a particular dataset as follows:

In [ ]:
ds = dset_dict['CMIP.CCCma.CanESM5.historical.Oyr.gn']
ds

Let's create a quick plot for a slice of the data:

In [ ]:
%matplotlib inline
ds.o2.isel(time=0, lev=0, member_id=5).plot()

We can execute more searches against the original catalog and/or against a subset of the original catalog:

In [ ]:
%%time
cat_fx = col.search(table_id='Ofx', grid_label='gn',
                    variable_id='volcello')
cat_fx.df.head()

In [ ]:
%%time
a = cat_fx.search(source_id="CESM2")
a.df.head()

In [ ]:
len(cat_fx.df), len(a.df)

## Using custom preprocessing functions
When comparing many models it is often necessary to preprocess (e.g. rename certain variables) them before running some analysis step. The `preprocess` argument lets the user pass a function, which is executed for each loaded zstore before merging.

In [ ]:
cat_pp = col.search(experiment_id=['historical'], table_id='Oyr', variable_id='o2', 
                    grid_label='gn', source_id=['IPSL-CM6A-LR', 'CanESM5'],
                    member_id='r10i1p1f1')
cat_pp.df

In [ ]:
# load the example
dset_dict_raw = cat_pp.to_dataset_dict(zarr_kwargs={'consolidated': True})
for k,ds in dset_dict_raw.items():
    print(k)
    print(list(ds.dims))

Note that both models follow a different naming scheme. We can define a little helper function and pass it to ` .to_dataset_dict` to fix this. For demonstration purposes we will focus on the vertical level dimension which is called `lev` in `CanESM5` and `olevel` in `IPSL-CM6A-LR`.

In [ ]:
def helper_func(ds):
    ds = ds.copy()
    # a short example 
    if 'olevel' in ds.dims: 
        ds = ds.rename({'olevel':'lev'})
    return ds
        

dset_dict_fixed = cat_pp.to_dataset_dict(zarr_kwargs={'consolidated': True}, preprocess=helper_func)
for k,ds in dset_dict_fixed.items():
    print(k)
    print(list(ds.dims))

This was just an example for one dimension. Check out [cmip6-preprocessing](https://github.com/jbusecke/cmip6_preprocessing/blob/master/notebooks/tutorial_intake_esm_preprocessing.ipynb) for a full renaming function for all available CMIP6 models and some other utilities.

In [ ]:
%load_ext watermark
%watermark -d -iv -p intake-esm -m -g -h